# Service

Service 是將一組Pods的網絡端口公開的方法，

Kubernetes為Pod提供提供一個cluster IP,並以Load Balancer方式提供服務。

在之前的教學裡，我們使用port forwarding方式access Pods IP，
可是Pods IP是會隨Pods的生命週期而銷毀、改變。

我們無法知道下一次由ReplicaSet或Deployment建立的新的Pods的IP，
所以需要借助Service，幫我們找出對應的endpoints


In [3]:
! kubectl delete deployment --all
! kubectl delete replicaset --all
! kubectl delete service --all
! kubectl delete pods --all

No resources found
No resources found
service "kubernetes" deleted
No resources found


In [2]:
# ! kubectl explain RESOURCE
! kubectl explain service

KIND:     Service
VERSION:  v1

DESCRIPTION:
     Service is a named abstraction of software service (for example, mysql)
     consisting of local port (for example 3306) that the proxy listens on, and
     the selector that determines which pods will answer requests sent through
     the proxy.

FIELDS:
   apiVersion	<string>
     APIVersion defines the versioned schema of this representation of an
     object. Servers should convert recognized schemas to the latest internal
     value, and may reject unrecognized values. More info:
     https://git.k8s.io/community/contributors/devel/sig-architecture/api-conventions.md#resources

   kind	<string>
     Kind is a string value representing the REST resource this object
     represents. Servers may infer this from the endpoint the client submits
     requests to. Cannot be updated. In CamelCase. More info:
     https://git.k8s.io/community/contributors/devel/sig-architecture/api-conventions.md#types-kinds

   metadata	<Object>
     Stand

In [ ]:
! cat deployment.yaml

In [27]:
# 透過範例 deployment.yaml 建立一個deployment
! kubectl delete -f deployment.yaml
! kubectl apply -f deployment.yaml

deployment.apps "hostname" deleted
deployment.apps/hostname created


In [28]:
! kubectl get deployment -o wide

NAME       READY   UP-TO-DATE   AVAILABLE   AGE   CONTAINERS   IMAGES                            SELECTOR
hostname   1/1     1            1           7s    hostname     docker.io/brokenpen/hostname:v1   app=hostname


In [29]:
! kubectl expose deployment/hostname --type=NodePort

Error from server (AlreadyExists): services "hostname" already exists


In [30]:
! kubectl get service hostname

NAME       TYPE       CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE
hostname   NodePort   10.107.81.175   <none>        8080:31066/TCP   5m38s


In [13]:
! kubectl get service hostname  -o wide

NAME       TYPE       CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE   SELECTOR
hostname   NodePort   10.107.81.175   <none>        8080:31066/TCP   36s   app=hostname


In [41]:
# 可以透過 kubectl patch 
# 設定 externalIP
# kubectl patch svc hostname -p '{"spec":{"externalIPs":["this-vm-ip"]}}'
! kubectl patch svc hostname -p '{"spec":{"externalIPs":["1.2.3.4"]}}'

service/hostname patched (no change)


In [42]:
! kubectl get service hostname  -o wide

NAME       TYPE       CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE   SELECTOR
hostname   NodePort   10.107.81.175   1.2.3.4       8080:31066/TCP   12m   app=hostname


In [51]:
! kubectl get pods --selector=app=hostname --show-labels

NAME                        READY   STATUS    RESTARTS   AGE   LABELS
hostname-85d476d6d4-klrvs   1/1     Running   0          10m   app=hostname,pod-template-hash=85d476d6d4


In [36]:
# miinkube ssh
# curl cluster-ip:8080
# curl localhost:31066 

In [45]:
# kubectl edit service hostname

In [53]:
# service 會負責建立endpoints,
# endpoints 只有目前readys 的Pod IP
! kubectl get endpoints hostname

NAME       ENDPOINTS         AGE
hostname   172.17.0.5:8080   15m


In [55]:
! kubectl scale deploy/hostname --replicas=2; echo;
! echo; sleep 5
! kubectl get endpoints hostname

deployment.apps/hostname scaled


NAME       ENDPOINTS                         AGE
hostname   172.17.0.3:8080,172.17.0.5:8080   16m


In [57]:
! kubectl get pods --selector=app=hostname -o wide

NAME                        READY   STATUS    RESTARTS   AGE   IP           NODE       NOMINATED NODE   READINESS GATES
hostname-85d476d6d4-klrvs   1/1     Running   0          10m   172.17.0.5   minikube   <none>           <none>
hostname-85d476d6d4-z8kbg   1/1     Running   0          24s   172.17.0.3   minikube   <none>           <none>


In [58]:
# 透過 minikube service 嘗試 access url
! minikube service hostname --url

http://192.168.49.2:31066


In [61]:
! curl $(minikube service hostname --url)

You've hit hostname-85d476d6d4-z8kbg


In [65]:
# 把目前的service刪掉
! kubectl delete service hostname 
# 透過 kubectl 建立 Service， 但不知道 --type
! kubectl expose deployment/hostname --port=8080 --target-port=8080

service "hostname" deleted
service/hostname exposed


In [67]:
# Default type 為 ClusterIP
# PORTS(s) 只有 cluster-ip 
! kubectl get service hostname -o wide

NAME       TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)    AGE   SELECTOR
hostname   ClusterIP   10.106.0.205   <none>        8080/TCP   41s   app=hostname


In [68]:
! kubectl get service hostname --no-headers -o custom-columns=":spec.clusterIP"

10.106.0.205


In [ ]:
# minikube ssh 
# curl cluster-ip

🤷  The control plane node "" does not exist.
👉  To start a cluster, run: "minikube start"


In [69]:
# 取得目前 hostname service 的yaml 輸出
! kubectl get service hostname -o yaml

apiVersion: v1
kind: Service
metadata:
  creationTimestamp: "2022-11-28T18:08:11Z"
  name: hostname
  namespace: default
  resourceVersion: "13384"
  uid: c03abc6d-0c13-4ab8-bb20-f0bcc722c5cb
spec:
  clusterIP: 10.106.0.205
  clusterIPs:
  - 10.106.0.205
  internalTrafficPolicy: Cluster
  ipFamilies:
  - IPv4
  ipFamilyPolicy: SingleStack
  ports:
  - port: 8080
    protocol: TCP
    targetPort: 8080
  selector:
    app: hostname
  sessionAffinity: None
  type: ClusterIP
status:
  loadBalancer: {}


In [70]:
# 把 hostname 的 service 刪掉
! kubectl delete service/hostname

service "hostname" deleted


In [73]:
# 列明目前的 hostname service
! kubectl get service hostname

Error from server (NotFound): services "hostname" not found


In [75]:
# 透過 dry-run 輸出 yaml
! kubectl expose deployment/hostname --port=8080 --target-port=8080 --dry-run -o yaml

W1128 18:10:33.101701  287378 helpers.go:663] --dry-run is deprecated and can be replaced with --dry-run=client.
apiVersion: v1
kind: Service
metadata:
  creationTimestamp: null
  name: hostname
spec:
  ports:
  - port: 8080
    protocol: TCP
    targetPort: 8080
  selector:
    app: hostname
status:
  loadBalancer: {}


In [72]:
# 透過 service.yaml 範例建立 service
! cat service.yaml

apiVersion: v1
kind: Service
metadata:
  name: hostname
spec:
  ports:
  - port: 8080
    protocol: TCP
    targetPort: 8080
  selector:
    app: hostname


In [77]:
! kubectl apply -f service.yaml

service/hostname created


In [78]:
! kubectl get service hostname -o wide

NAME       TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE   SELECTOR
hostname   ClusterIP   10.106.93.211   <none>        8080/TCP   1s    app=hostname


In [79]:
! kubectl get po -o wide 

NAME                        READY   STATUS    RESTARTS   AGE     IP           NODE       NOMINATED NODE   READINESS GATES
hostname-85d476d6d4-klrvs   1/1     Running   0          16m     172.17.0.5   minikube   <none>           <none>
hostname-85d476d6d4-z8kbg   1/1     Running   0          6m27s   172.17.0.3   minikube   <none>           <none>


In [80]:
! kubectl get po -o=custom-columns='IP:.status.podIP' --no-headers

172.17.0.5
172.17.0.3


In [81]:
# minikube ssh
# curl 172.17.0.5
# curl 172.17.0.3

curl: (7) Failed to connect to 172.17.0.5 port 80 after 3071 ms: No route to host
curl: (7) Failed to connect to 172.17.0.3 port 8080 after 3075 ms: No route to host


In [87]:
! echo CLUSTER-IP: $(kubectl get service hostname -o=custom-columns='IP:.spec.clusterIP' --no-headers)

CLUSTER-IP: 10.106.93.211


In [83]:
# minikube ssh 
# curl cluster-ip:8080

^C


In [88]:
! kubectl delete deploy --all
! kubectl delete replicaset --all
! kubectl delete service --all
! kubectl delete pod --all

deployment.apps "hostname" deleted
No resources found
service "hostname" deleted
service "kubernetes" deleted
pod "hostname-85d476d6d4-klrvs" deleted
pod "hostname-85d476d6d4-z8kbg" deleted
